In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'scalar_coupling_contributions.csv', 'structures', 'structures.csv', 'test.csv', 'magnetic_shielding_tensors.csv', 'mulliken_charges.csv', 'potential_energy.csv', 'dipole_moments.csv']


In [22]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')


In [23]:
train_df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [24]:
structures = pd.read_csv('../input/structures.csv')

def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

train_df = map_atom_info(train_df, 0)
train_df = map_atom_info(train_df, 1)

test_df = map_atom_info(test_df, 0)
test_df = map_atom_info(test_df, 1)

In [25]:
train_df.tail()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
4658142,4658142,dsgdb9nsd_133884,17,4,2JHC,3.543450,H,1.12655,-1.348733,-1.933838,C,1.629865,-0.747236,0.235262
4658143,4658143,dsgdb9nsd_133884,17,5,3JHC,0.568997,H,1.12655,-1.348733,-1.933838,C,1.415947,0.620773,0.939122
4658144,4658144,dsgdb9nsd_133884,17,6,3JHC,1.173370,H,1.12655,-1.348733,-1.933838,C,-0.027076,0.747033,0.478506
4658145,4658145,dsgdb9nsd_133884,17,7,2JHC,4.762010,H,1.12655,-1.348733,-1.933838,C,-0.131901,0.356983,-1.010196
4658146,4658146,dsgdb9nsd_133884,17,8,1JHC,117.934000,H,1.12655,-1.348733,-1.933838,C,0.787756,-0.840138,-1.042152


In [26]:
scalar_df = pd.read_csv('../input/scalar_coupling_contributions.csv')
scalar_df.tail()

,molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso,dso
4658142,dsgdb9nsd_133884,17,4,2JHC,3.586440,0.019741,0.150477,-0.213205
4658143,dsgdb9nsd_133884,17,5,3JHC,0.674583,-0.007276,0.305078,-0.403388
4658144,dsgdb9nsd_133884,17,6,3JHC,1.337470,-0.028423,0.312240,-0.447920
4658145,dsgdb9nsd_133884,17,7,2JHC,4.800620,0.139202,-0.053102,-0.124710
4658146,dsgdb9nsd_133884,17,8,1JHC,115.975000,0.378277,0.450213,1.130600



#arranging and merging the scalar coupling coefficient using same method

def map_scalar_atom_info(df):
    df = pd.merge(df, scalar_df, how = 'left',
                  left_on  = ['molecule_name', 'atom_index_0','atom_index_1','type'],
                  right_on = ['molecule_name',  'atom_index_0','atom_index_1','type'])
    

    return df

train_df = map_scalar_atom_info(train_df)

test_df = map_scalar_atom_info(test_df)

In [29]:
test_df.tail()

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
2505537,7163684,dsgdb9nsd_133885,15,3,2JHC,H,1.260745,-1.246754,-1.906767,C,0.271030,-1.618614,0.144805
2505538,7163685,dsgdb9nsd_133885,15,4,2JHC,H,1.260745,-1.246754,-1.906767,C,1.528454,-0.755123,0.339508
2505539,7163686,dsgdb9nsd_133885,15,6,3JHC,H,1.260745,-1.246754,-1.906767,C,-0.057400,0.611210,0.541102
2505540,7163687,dsgdb9nsd_133885,15,7,2JHC,H,1.260745,-1.246754,-1.906767,C,-0.095929,0.380424,-0.972098
2505541,7163688,dsgdb9nsd_133885,15,8,1JHC,H,1.260745,-1.246754,-1.906767,C,0.816694,-0.813067,-1.022360


In [30]:
train_df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001


In [34]:
train_df['atom_1'].unique()

array(['C', 'H', 'N'], dtype=object)

In [35]:
train_df['atom_0'].unique()

array(['H'], dtype=object)

In [36]:
test_df['atom_0'].unique()

array(['H'], dtype=object)

In [37]:
test_df['atom_1'].unique()

array(['C', 'H', 'N'], dtype=object)

In [38]:
train_df['type'].unique()

array(['1JHC', '2JHH', '1JHN', '2JHN', '2JHC', '3JHH', '3JHC', '3JHN'],
      dtype=object)

In [39]:
test_df['type'].unique()

array(['2JHC', '1JHC', '3JHH', '3JHC', '2JHH', '1JHN', '3JHN', '2JHN'],
      dtype=object)

In [41]:
#replace atom_0 and atom_1 
#C:0,H:1,N:2

train_df=train_df.replace({'atom_0': {'H': 0, 'C': 1,'N':2}})
train_df=train_df.replace({'atom_1': {'H': 0, 'C': 1,'N':2}})

#test set

test_df=test_df.replace({'atom_0': {'H': 0, 'C': 1,'N':2}})
test_df=test_df.replace({'atom_1': {'H': 0, 'C': 1,'N':2}})



#replace type with 
# '2JHC':0, '1JHC':1, '3JHH':2, '3JHC':3, '2JHH':4, '1JHN':5, '3JHN:6', '2JHN':7

train_df=train_df.replace({'type': {'2JHC':0, '1JHC':1, '3JHH':2, '3JHC':3, '2JHH':4, '1JHN':5, '3JHN':6, '2JHN':7}})
test_df=test_df.replace({'type': {'2JHC':0, '1JHC':1, '3JHH':2, '3JHC':3, '2JHH':4, '1JHN':5, '3JHN':6, '2JHN':7}})


In [48]:
#molecules to be test names are store in separate file
molecules_list = test_df['molecule_name']
molecules_list.head(2)

0    dsgdb9nsd_000004
1    dsgdb9nsd_000004
Name: molecule_name, dtype: object

In [49]:
train_df = train_df.drop('molecule_name',axis=1)
train_df = train_df.drop('id',axis=1)

test_df =  test_df.drop('molecule_name',axis=1)
test_df = test_df.drop('id',axis=1)

train_df.head()

,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,1,0,1,84.8076,0,0.002150,-0.006031,0.001976,1,-0.012698,1.085804,0.008001
1,1,2,4,-11.2570,0,0.002150,-0.006031,0.001976,0,1.011731,1.463751,0.000277
2,1,3,4,-11.2548,0,0.002150,-0.006031,0.001976,0,-0.540815,1.447527,-0.876644
3,1,4,4,-11.2543,0,0.002150,-0.006031,0.001976,0,-0.523814,1.437933,0.906397
4,2,0,1,84.8074,0,1.011731,1.463751,0.000277,1,-0.012698,1.085804,0.008001


In [51]:
test_df.head()

,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,2,0,0,0,-1.661639,0.0,1.0,1,0.599539,0.0,1.0
1,2,1,1,0,-1.661639,0.0,1.0,1,-0.599539,0.0,1.0
2,2,3,2,0,-1.661639,0.0,1.0,0,1.661639,0.0,1.0
3,3,0,1,0,1.661639,0.0,1.0,1,0.599539,0.0,1.0
4,3,1,0,0,1.661639,0.0,1.0,1,-0.599539,0.0,1.0


In [53]:
train_df.shape

(4658147, 12)

In [54]:
test_df.shape

(2505542, 11)

In [56]:
#taking y value out from train dataframes
y_df = train_df['scalar_coupling_constant']

#removing scalar_coupling_constant from train dataframes
train_df= train_df.drop('scalar_coupling_constant',axis=1)

In [58]:
train_df.shape,test_df.shape,y_df.shape

((4658147, 11), (2505542, 11), (4658147,))

In [59]:
#convert into numpy array
x_train = np.array(train_df,dtype='float32')
y_train = np.array(y_df,dtype='float32')
x_test = np.array(test_df,dtype='float32')

x_train.shape,y_train.shape,x_test.shape

((4658147, 11), (4658147,), (2505542, 11))

In [60]:
#spilit data for validation and cross check only for 20%

from sklearn.model_selection import train_test_split
(x_train,x_validate,y_train,y_validate) = train_test_split(
    x_train,y_train,test_size=0.2,random_state=12345
)

In [61]:
x_train.shape,x_validate.shape

((3726517, 11), (931630, 11))

In [64]:
#normalise the data

from sklearn import preprocessing

x_train = preprocessing.scale(x_train)
x_validate = preprocessing.scale(x_validate)
x_test = preprocessing.scale(x_test)





/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:197: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:197: UserWarning: Numerical issues were enc

In [65]:
x_train.shape,x_validate.shape,x_test.shape,y_train.shape,y_validate.shape

((3726517, 11), (931630, 11), (2505542, 11), (3726517,), (931630,))

In [66]:
#models

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier


In [70]:
#error occured Unknown label type: 'continous'
#solution found from https://www.kaggle.com/pratsiuk/valueerror-unknown-label-type-continuous
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
y_train_encoded = lab_enc.fit_transform(y_train)
y_validate_encoded = lab_enc.fit_transform(y_validate)
print(y_train_encoded)
print(utils.multiclass.type_of_target(y_train))
print(utils.multiclass.type_of_target(y_train.astype('int')))
print(utils.multiclass.type_of_target(y_train_encoded))

[ 986490  958513  562677 ...  776977 1009294  998621]
continuous
multiclass
multiclass


In [ ]:
#1 model logistic regression
logreg = LogisticRegression()
logreg.fit(x_train, y_train_encoded)
y_pred_lr = logreg.predict(x_test)
print('Training Accuracy for logistic regression is : { } '.format(logreg.score(x_train, y_train_encoded)))
print('Testing Accuracy for logistic regression is : { } '.format(logreg.score(x_validate, y_validate_encoded)))


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
